In [1]:
import pandas as pd
import numpy as np
import json
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdChemReactions
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.Draw import IPythonConsole
from IPython.display import SVG, display
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Chem import MACCSkeys
import pandas as pd
import os
import itertools
import pickle
import sys
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp
from multiprocessing import Pool

from tqdm import tqdm 
import cobra
from itertools import chain, combinations
import sys
import multiprocessing
from functools import partial
tqdm.pandas()
# sys.path.append('../Code/')
sys.path.append('../')

from common import *

In [2]:
#input
num = 50
cut_off = 0.3
cut_off_retrosynthesis_path = f'../../../Data_retrosynthesis/not_lipid/top{num}_{cut_off}_add_no_ec_re/'
cut_off_path = f'../../../Results/not_lipid/top{num}_{cut_off}_add_no_ec_re/'

# path = '../Data/output/not_lipid/retrosynthesis/top40_0.3/'
not_lipid_rxndb_path = cut_off_retrosynthesis_path + 'RXNdb_top50_0.3/'
not_lipid_YMDB_fail_met_smile_uptake_file = cut_off_path + f'YMDB_fail_met_smile_uptake_top{num}_{cut_off}.pickle'
total_met_inchikey0_file = '../../../Data/total_met_inchikey0.pickle'
YMDB_fail_met_smile_file = cut_off_path + f'YMDB_fail_met_smile_top{num}_{cut_off}.pickle'
uptake_met_path = '../../../Data/ymdb/ymdb_uptake.csv'

# not_lipid_rxndb_all_no_drop_path = cut_off_path + 'RXNdb_all_no_drop_top50.json'
rxndb_path = cut_off_retrosynthesis_path + f'RXNdb_top{num}_{cut_off}/'


#output
not_lipid_rxndb_inteme_drop_duplicate_path = cut_off_path + f'RXNdb_all_top50_0.3_all.csv'
not_lipid_fail_target_rxndb_path = cut_off_retrosynthesis_path + 'RXNdb_fail_target/'
not_lipid_rxndb_inteme_only_one_new_path = cut_off_path + 'RXNdb_inteme_only_one_new_top50_drop_unvalid.csv'
not_lipid_rxndb_inteme_only_one_new_target_path = cut_off_path + 'RXNdb_inteme_only_one_new_target_top50.csv'

sink0_path = cut_off_path + 'sink0_new_met_only_new_one_top50.pickle'


In [13]:
if not os.path.exists(not_lipid_fail_target_rxndb_path):
    os.makedirs(not_lipid_fail_target_rxndb_path)

In [3]:
def process_file_mer(file,rxndb_path,inchikey0,rxndb_drop_path):
    with open(rxndb_path + file, 'r') as f:
        rxndb = json.load(f)    
    remove = []
    for k,v in rxndb.items():
        
        productsmile = v['productsmile'].split('.')
        if any([smiles2inchikey0(p) in inchikey0  for p in productsmile]):
            pass
        
        else:
            remove.append(k)
    for k in remove:
        rxndb.pop(k)
    with open(rxndb_drop_path + file, 'w') as f:
        json.dump(rxndb, f,indent=4)

def drop_rxndb_mer(rxndb_path, rxndb_drop_path, inchikey0,num_processes=60):
    files = os.listdir(rxndb_path)
    process_file_partial = partial(process_file_mer,rxndb_path=rxndb_path,inchikey0=inchikey0,rxndb_drop_path=rxndb_drop_path)
    with mp.Pool(num_processes) as p:
        list(tqdm(p.imap_unordered(process_file_partial, files), total=len(files)))

In [15]:
not_lipid_YMDB_fail_met_smile = pickle.load(open(not_lipid_YMDB_fail_met_smile_uptake_file, 'rb'))
not_lipid_YMDB_fail_met_inchikey0 = [smiles2inchikey0(smile) for smile in not_lipid_YMDB_fail_met_smile ]

In [16]:
drop_rxndb_mer(not_lipid_rxndb_path, not_lipid_fail_target_rxndb_path, not_lipid_YMDB_fail_met_inchikey0,num_processes=50)

100%|██████████| 16955/16955 [04:30<00:00, 62.79it/s] 


In [4]:
def process_file_drop_duplicate(file,rxndb_path):
    try:
        with open(rxndb_path + file, 'r') as f:
            rxndb = json.load(f)    
    
        if len(rxndb) != 0:
            rxndb_df = pd.DataFrame(rxndb).transpose()
            
            rxndb_df['reactant_inchikey0'] = rxndb_df['reactant_smile'].apply(lambda x: '.'.join(sorted([smiles2inchikey0(i) for i in (x.split('.'))])))
            rxndb_df['product_inchikey0'] = rxndb_df['productsmile'].apply(lambda x: '.'.join(sorted([smiles2inchikey0(i) for i in (x.split('.'))])))
            # rxndb_df['product_inchikey0'] = rxndb_df['product_inchikey0'].apply(lambda x: '.'.join(sorted(x.split('.'))))
            # rxndb_df['reactant_inchikey0'] = rxndb_df['reactant_inchikey0'].apply(lambda x: '.'.join(sorted(x.split('.'))))
            rxndb = rxndb_df.to_dict(orient='index')
            with open(rxndb_path + file, 'w') as f:
                json.dump(rxndb, f,indent=4)
    except:
        print(file)
        pass

def process_file_drop_duplicate_parallel(rxndb_path,num_processes=50):
    files = os.listdir(rxndb_path)
#     files = ['RXNdb_14838.json','RXNdb_7284.json','RXNdb_10683.json','RXNdb_10627.json',
# 'RXNdb_17365.json','RXNdb_13669.json','RXNdb_14857.json','RXNdb_15735.json','RXNdb_5346.json','RXNdb_7276.json']
    process_file_partial = partial(process_file_drop_duplicate,rxndb_path=rxndb_path)
    with mp.Pool(num_processes) as p:
        list(tqdm(p.imap_unordered(process_file_partial, files), total=len(files)))



In [5]:
def drop_duplicate_rxndb(not_lipid_rxndb_inteme_drop_duplicate_path,rxndb_path):
    files = os.listdir(rxndb_path) 
    with open(rxndb_path + files[0], 'r') as f:
        rxndb_total = json.load(f)
        rxndb_total_df = pd.DataFrame(rxndb_total).transpose()
        # rxndb_total_df['reactant_inchikey0'] = rxndb_total_df['reactant_smile'].apply(lambda x: '.'.join(sorted([smiles2inchikey0(i) for i in (x.split('.'))])))
        # rxndb_total_df['product_inchikey0'] = rxndb_total_df['productsmile'].apply(lambda x: '.'.join(sorted([smiles2inchikey0(i) for i in (x.split('.'))])))
        
    for file in tqdm(files[1:],total=len(files[1:])):
        with open(rxndb_path + file, 'r') as f:
            rxndb = json.load(f) 
    
        rxndb_df = pd.DataFrame(rxndb).transpose()
        # rxndb_df['reactant_inchikey0'] = rxndb_df['reactant_smile'].apply(lambda x: '.'.join(sorted([smiles2inchikey0(i) for i in (x.split('.'))])))
        # rxndb_df['product_inchikey0'] = rxndb_df['productsmile'].apply(lambda x: '.'.join(sorted([smiles2inchikey0(i) for i in (x.split('.'))])))
        rxndb_total_df = pd.concat([rxndb_total_df,rxndb_df],axis=0)
    rxndb_total_df = rxndb_total_df.drop_duplicates(subset=['reactant_inchikey0', 'product_inchikey0'], keep='first').reset_index(drop=True)
    rxndb_total_df.to_csv(not_lipid_rxndb_inteme_drop_duplicate_path )
    print(rxndb_total_df.shape)

In [19]:
process_file_drop_duplicate_parallel(not_lipid_fail_target_rxndb_path,num_processes=30)


100%|██████████| 16955/16955 [03:11<00:00, 88.52it/s] 


In [6]:
drop_duplicate_rxndb(not_lipid_rxndb_inteme_drop_duplicate_path,not_lipid_fail_target_rxndb_path)

  0%|          | 0/16954 [00:00<?, ?it/s]

100%|██████████| 16954/16954 [56:12<00:00,  5.03it/s] 


(126778, 11)


In [7]:

def choose_only_one_new(not_lipid_rxndb_inteme_drop_duplicate_path,total_inchikey0,not_lipid_rxndb_inteme_only_one_new_path):
    rxndb_total_df = pd.read_csv(not_lipid_rxndb_inteme_drop_duplicate_path)
    rxndb_total_df['new_num'] = None
    rxndb_total_df['reactant_inchikey0'] = rxndb_total_df['reactant_inchikey0'].apply(lambda x: x.split('.'))
    rxndb_total_df['new_num'] = rxndb_total_df['reactant_inchikey0'].apply(lambda x: sum([1 for i in x if i not in total_inchikey0 ]))
    rxndb_total_df = rxndb_total_df[rxndb_total_df['new_num'] == 1]
    rxndb_total_df.to_csv(not_lipid_rxndb_inteme_only_one_new_path)



In [8]:
total_inchikey0 = load_pickle(total_met_inchikey0_file)
choose_only_one_new(not_lipid_rxndb_inteme_drop_duplicate_path,total_inchikey0,not_lipid_rxndb_inteme_only_one_new_path)

In [9]:
def filter_df(df):
    for index, row in df.iterrows():
        reactant_smile = row['reactant_smile'].split('.')
        for i in reactant_smile:
            if not Chem.MolFromSmiles(i):
                df.drop(index, inplace=True)
                break
    for index, row in df.iterrows():
        productsmile = row['productsmile'].split('.')
        for i in productsmile:
            if not Chem.MolFromSmiles(i):
                df.drop(index, inplace=True)
                break
    return df

In [10]:
not_lipid_rxndb_inteme_only_one_new = pd.read_csv(not_lipid_rxndb_inteme_only_one_new_path)
print(not_lipid_rxndb_inteme_only_one_new.shape)

(106869, 14)


In [11]:
chunks = np.array_split(not_lipid_rxndb_inteme_only_one_new, 50)
with mp.Pool(50) as pool:
    results = list(tqdm(pool.imap(filter_df, chunks), total=len(chunks)))
rxndb_total_df = pd.concat(results)
rxndb_total_df.shape

100%|██████████| 50/50 [00:05<00:00,  8.54it/s]


(106740, 14)

In [12]:
rxndb_total_df.to_csv(not_lipid_rxndb_inteme_only_one_new_target_path,index=False)